# Lesson 6: Expanding the SEC Knowledge Graph

### Import packages and set up Neo4j

In [1]:
from dotenv import load_dotenv
import os

# Load from environment
load_dotenv(".env", override=True)

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE") or "neo4j"

# Gloabal constants
VECTOR_INDEX_NAME = "form_10k_chunks"
VECTOR_NODE_LABEL = "Chunk"
VECTOR_SOURCE_PROPERTY = "text"
VECTOR_EMBEDDING_PROPERTY = "textEmbedding"

In [2]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

In [3]:
import textwrap

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

In [36]:
kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

### Read the collection of Form 13s
- Investment management firms must report on their investments in companies to the SEC by filing a document called **Form 13**
- You'll load a collection of Form 13 for managers that have invested in NetApp
- You can check out the CSV file by navigating to the data directory using the File menu at the top of the notebook

In [5]:
import csv

all_form13 = []

with open(file="./data/form13.csv", mode="r") as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for row in csv_reader: # each row will be a dictionary
        all_form13.append(row)

- Look at the contents of Form 13

In [6]:
len(all_form13)

561

In [7]:
all_form13[0:5]

[{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
  'managerCik': '1000275',
  'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'managerName': 'Royal Bank of Canada',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '64395000000.0',
  'shares': '842850'},
 {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt',
  'managerCik': '1002784',
  'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805',
  'managerName': 'SHELTON CAPITAL MANAGEMENT',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '2989085000.0',
  'shares': '39124'},
 {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt',
  'managerCik': '1007280',
  'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215',
  'managerName': 'PUBLIC 

### Create company nodes in the graph
- Use the companies identified in the Form 13s to create `Company` nodes
- For now, there is only one company - NetApp

In [8]:
# work with just the first row for now
first_form13 = all_form13[0]

cypher = """
    MERGE (com:Company {cusip6: $cusip6})
    ON CREATE
    SET com.companyName = $companyName,
        com.cusip = $cusip
"""

kg.query(query=cypher,
         params={"cusip6": first_form13["cusip6"],
                 "companyName": first_form13["companyName"],
                 "cusip": first_form13["cusip"]})

[]

In [9]:
cypher = """
    MATCH (com: Company)
    RETURN com LIMIT 1
"""

kg.query(query=cypher)

[{'com': {'cusip': '64110D104',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

- Update the company name to match Form 10-K

In [10]:
cypher = """
    MATCH (com: Company), (form:Form)
        WHERE com.cusip6 = form.cusip6
    RETURN com.companyName, form.names
"""

kg.query(query=cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}]

In [11]:
cypher = """
    MATCH (com: Company), (form:Form)
        WHERE com.cusip6 = form.cusip6
    SET com.names = form.names
"""

kg.query(query=cypher)

[]

- Create a `FILED` relationship between the company and the Form 10-K node

In [12]:
cypher = """
    MATCH (com: Company), (form: Form)
    WHERE com.cusip6 = form.cusip6
    MERGE (com)-[:FILED]->(form)
"""

kg.query(query=cypher)

[]

### Create manager nodes
- Create a `manager` node for companies that have filed a Form 13 to report their investment in NetApp
- Start with the single manager who filed the first Form 13 in the list

In [13]:
cypher = """
    MERGE (mgr:Manager {managerCik:$managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

kg.query(query=cypher, params={"managerParam": first_form13})

[]

Check the insertion of `Manager` node

In [14]:
cypher = """
    MATCH (mgr: Manager)
    RETURN mgr LIMIT 1
"""

kg.query(query=cypher)

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

- Create a uniqueness constraint to avoid duplicate managers

In [15]:
cypher = """
    CREATE CONSTRAINT unique_manager
    IF NOT EXISTS
    FOR (n:Manager)
    REQUIRE n.managerCik IS UNIQUE
"""

kg.query(query=cypher)

[]

- Create a fulltext index of manager names to enable text search

In [16]:
cypher = """
    CREATE FULLTEXT INDEX fullTextManagerNames
    IF NOT EXISTS
    FOR (mgr: Manager)
    ON EACH [mgr.managerName]
"""

kg.query(query=cypher)

[]

Now search on full text index of manager names

In [18]:
cypher = """
    CALL db.index.fulltext.queryNodes(
        "fullTextManagerNames", "royal bank")
        YIELD node, score
    RETURN node.managerName, score
"""

kg.query(query=cypher)

[{'node.managerName': 'Royal Bank of Canada', 'score': 0.2615291476249695}]

- Create nodes for all companies that filed a Form 13

In [19]:
cypher = """
    MERGE (mgr:Manager {managerCik:$managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

# Loop through all Form 13s
for form13 in all_form13:
    kg.query(query=cypher, params={"managerParam": form13})

Check the manager nodes

In [20]:
cypher = """
    MATCH (mgr: Manager)
    RETURN COUNT (mgr)
"""

kg.query(query=cypher)

[{'COUNT (mgr)': 561}]

### Create relationships between managers and companies
- Match companies with managers based on data in the Form 13
- Create an `OWNS_STOCK_IN` relationship between the manager and the company
- Start with the single manager who filed the first Form 13 in the list

In [21]:
cypher = """
    MATCH (mgr: Manager {managerCik: $investmentParam.managerCik}),
        (com: Company {cusip6: $investmentParam.cusip6})
    RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(query=cypher, params={"investmentParam": first_form13})

[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [23]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
        (com: Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter}]->(com)
ON CREATE
    SET owns.value = toFloat($ownsParam.value),
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(query=cypher, params={"ownsParam": first_form13})

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

Now check for the added relationship in the database

In [24]:
cypher = """
    MATCH (mgr: Manager {managerCik: $ownsParam.managerCik})-
    [owns: OWNS_STOCK_IN]->(com: Company {cusip6: $ownsParam.cusip6})
    RETURN owns {.shares, .value}
"""

kg.query(query=cypher, params={"ownsParam": first_form13})

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

- Create relationships between all of the managers who filed Form 13s and the company

In [25]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
        (com: Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter}]->(com)
ON CREATE
    SET owns.value = toFloat($ownsParam.value),
        owns.shares = toInteger($ownsParam.shares)
"""

# Loop through all Form 13s
for form13 in all_form13:
    kg.query(query=cypher, params={"ownsParam": form13})

Count the number of relationship `OWNS_STOCK_IN`

In [26]:
cypher = """
    MATCH (:Manager)-[owns: OWNS_STOCK_IN]->(:Company)
    RETURN count(owns) as investments
"""

kg.query(query=cypher)

[{'investments': 561}]

In [27]:
kg.refresh_schema()

print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


### Determine the number of investors
- Start by finding a form 10-K chunk, and save to use in subsequent queries

In [30]:
cypher = """
    MATCH (chunk: Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
"""

chunk_rows = kg.query(query=cypher)
print(chunk_rows)

[{'chunkId': '0000950170-23-027948-item1-chunk0000'}]


In [31]:
chunk_first_row = chunk_rows[0]
print(chunk_first_row)

{'chunkId': '0000950170-23-027948-item1-chunk0000'}


In [32]:
ref_chunk_id = chunk_first_row["chunkId"]
ref_chunk_id

'0000950170-23-027948-item1-chunk0000'

- Build a path from Form 10-K chunk to companies and managers

In [33]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f: Form)
    RETURN f.source
"""

kg.query(query=cypher, params={"chunkIdParam": ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [34]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f: Form),
            (com:Company)-[:FILED]->(f)
    RETURN com.companyName as name
"""

kg.query(query=cypher, params={"chunkIdParam": ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [35]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f: Form),
            (com: Company)-[:FILED]->(f),
            (mgr: Manager)-[:OWNS_STOCK_IN]->(com)
    RETURN com.companyName, count(mgr) as numberOfInvestors 
"""

kg.query(query=cypher, params={"chunkIdParam": ref_chunk_id})

[{'com.companyName': 'NETAPP INC', 'numberOfInvestors': 561}]

### Use queries to build additional context for LLM
- Create sentences that indicate how much stock a manager has invested in a company

In [39]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f: Form),
            (com: Company)-[:FILED]->(f),
            (mgr: Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares + " shares of " + com.companyName + " at a value of $" + 
            apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
"""

kg.query(query=cypher, params={"chunkIdParam": ref_chunk_id})

[{'text': 'Royal Bank of Canada owns 842850 shares of NETAPP INC at a value of $64,395,000,000'},
 {'text': 'SHELTON CAPITAL MANAGEMENT owns 39124 shares of NETAPP INC at a value of $2,989,085,000'},
 {'text': 'PUBLIC EMPLOYEES RETIREMENT SYSTEM OF OHIO owns 106941 shares of NETAPP INC at a value of $8,170,000'},
 {'text': 'WILBANKS SMITH & THOMAS ASSET MANAGEMENT LLC owns 6617 shares of NETAPP INC at a value of $505,539,000'},
 {'text': 'DEPRINCE RACE & ZOLLO INC owns 320581 shares of NETAPP INC at a value of $24,492,389,000'},
 {'text': 'COMMERCE BANK owns 101422 shares of NETAPP INC at a value of $7,748,640,000'},
 {'text': 'D. E. Shaw & Co., Inc. owns 323440 shares of NETAPP INC at a value of $24,710,816,000'},
 {'text': 'EDMP, INC. owns 4698 shares of NETAPP INC at a value of $358,927,000'},
 {'text': 'NATIXIS ADVISORS, L.P. owns 40296 shares of NETAPP INC at a value of $3,079,000'},
 {'text': 'Smithfield Trust Co owns 215 shares of NETAPP INC at a value of $16,000'}]

In [40]:
results = kg.query(query=cypher, params={"chunkIdParam": ref_chunk_id})

print(textwrap.fill(text=results[0]["text"], width=60))

Royal Bank of Canada owns 842850 shares of NETAPP INC at a
value of $64,395,000,000


- Create a plain Question Answer chain
- Similarity serch only, no augmentation by Cypher query

In [41]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY
)

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever
)

- Create a second QA chain
- Augment similarity search using sentences found by the investment query above

In [42]:
investment_retrieval_query = """
    MATCH (node)-[:PART_OF]->(f: Form),
            (f)<-[:FILED]-(com: Company),
            (com)-[owns:OWNS_STOCK_IN]-(mgr: Manager)
    WITH node, score, mgr, owns, com
        ORDER by owns.shares DESC LIMIT 10
    WITH COLLECT (
        mgr.managerName + " owns " + owns.shares + " shares of " + com.companyName +
             " at a value of $" + apoc.number.format(toInteger(owns.value))
    ) AS investment_statements, node, score
    RETURN apoc.text.join(investment_statements, "\n") + 
        "\n" + node.text as text,
        score, 
        {
            source: node.source
        } as metadata
"""

In [43]:
vector_store_with_investment = Neo4jVector.from_existing_index(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever_with_investments
)

- Compare the outputs!

In [44]:
question = "In a single sentence, tell me about Netapp."

In [46]:
plain_chain({"question": question},
            return_only_outputs=True)

{'answer': 'NetApp is a global cloud-led, data-centric software company that enables applications to run faster, more reliably, and more securely across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [47]:
investment_chain({"question": question},
                 return_only_outputs=True)

{'answer': 'NetApp is a global cloud-led, data-centric software company that enables applications to run faster, more reliably, and more securely across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

- The LLM didn't make use of the investor information since the question didn't ask about investors
- Change the question and ask again

In [48]:
question = "In a single sentence, tell me about Netapp investors."

In [49]:
plain_chain(inputs={"question": question},
            return_only_outputs=True)

{'answer': "NetApp investors are critical to the company's growth and success.\n",
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [50]:
investment_chain(inputs={"question": question},
                 return_only_outputs=True)

{'answer': 'VANGUARD GROUP INC, BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/ are investors in Netapp.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

### Try for yourself

- Try changing the query above to retrieve other information
- Try asking different questions
- Note, if you change the Cypher query, you'll need to reset the retriever and QA chain

In [51]:
question = "In a single sentence, tell me about Netapp investors from New York."

In [52]:
plain_chain(inputs={"question": question},
            return_only_outputs=True)

{'answer': 'There is no information about Netapp investors from New York in the provided content.\n',
 'sources': ''}

In [53]:
investment_chain(inputs={"question": question},
                 return_only_outputs=True)

{'answer': 'VANGUARD GROUP INC, BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/ are Netapp investors from New York.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

Check for the validity of the above answer by `investment_chain`

In [58]:
cypher = """
    MATCH (mgr: Manager {managerName: $managerNameParam})
    RETURN mgr.managerName, mgr.managerAddress
"""

for managerName in ["VANGUARD GROUP INC", "BlackRock Inc.", "PRIMECAP MANAGEMENT CO/CA/"]:  
    print(kg.query(query=cypher, params={"managerNameParam": managerName}))

[{'mgr.managerName': 'VANGUARD GROUP INC', 'mgr.managerAddress': 'Po Box 2600, V26, Valley Forge, PA, 19482-2600'}]
[{'mgr.managerName': 'BlackRock Inc.', 'mgr.managerAddress': '50 Hudson Yards, New York, NY, 10001'}]
[{'mgr.managerName': 'PRIMECAP MANAGEMENT CO/CA/', 'mgr.managerAddress': '177 EAST COLORADO BLVD., 11TH FLOOR, PASADENA, CA, 91105'}]


Only 1 out of 3 is correct

In [59]:
question = "Has BlackRock invested in Netapp?"

In [60]:
plain_chain(inputs={"question": question},
            return_only_outputs=True)

{'answer': 'BlackRock has not invested in NetApp.\n', 'sources': ''}

In [61]:
investment_chain(inputs={"question": question},
                 return_only_outputs=True)

{'answer': 'BlackRock Inc. owns 18241407 shares of NETAPP INC.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}